# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902066


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:02<00:42,  1.46s/it]

Rendering models:  13%|█▎        | 4/31 [00:03<00:28,  1.05s/it]

Rendering models:  19%|█▉        | 6/31 [00:07<00:35,  1.41s/it]

Rendering models:  23%|██▎       | 7/31 [00:07<00:25,  1.06s/it]

...

Rendering models:  90%|█████████ | 28/31 [00:10<00:00,  7.48it/s]

Rendering models:  94%|█████████▎| 29/31 [00:10<00:00,  7.62it/s]

Rendering models:  97%|█████████▋| 30/31 [00:10<00:00,  7.68it/s]

Rendering models: 100%|██████████| 31/31 [00:10<00:00,  7.38it/s]

equidad1                              0.000085
kayleebug2017                         0.000168
ElisabethB                            0.000049
not-logged-in-c66140310d6a5859bd6c    0.001353
not-logged-in-8c0c44faf754d344b18a    0.002357
not-logged-in-05ca4a4e98bc7b8f1b53    1.734438
Thedunkmasta                          0.000165
not-logged-in-8c0e6d871dcd753b1e81    0.000184
not-logged-in-7526601586eaa938b3ee    0.000072
mlang2017                             0.000123
Alexander00                           0.000054
lsautter                              0.000055
AdamMorse                             0.000064
pangeli5                              0.000082
Saharisunshine                        0.000125
RaulE                                 0.000090
Lavadude                              0.014650
w7250369                              0.000165
LeusaneLordelo                        0.000178
slambier                              0.000042
not-logged-in-3a93234a2311f6df1afd    0.000060
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())